# Tesing BasicCatalogFilter 

The BasicCatalogFilter makes it possible to accept or reject alerts if there is a spatial coincidence with some sources of a given astronomical catalog. In this notebook, we illustrate how to use the BasicCatalogFilter to reject alerts nearby AGNs catalogued in the Million Quasar Catalog ([MQC](http://quasars.org/milliquas.htm))


For this example, we define a channel named HU_NOAGN by providing the channel configuration parameters. The way our example channel HU_NOAGN interacts with the BasicCatalogFilter is defined in a specific section of the channel config. In particular, you have to provide: 

* name of the catalog to be used for cross matching.
* name of the columns of the catalog containing Ra and Dec (e.g. 'RAJ2000', 'ra', ecc.)
* search radius (in arcsec)
* how the position of the alert has to be estimated, e.g. the all-epoch average or the current position of the alert. 
* search method for the catalog queries. This depends on the chosen indexing strategy for the catalog database.
* what to do when a match has been found within the search radius, i.e. either reject or accept the alert.

Here is the configuration of the BasicCatalogFilter we will demonstrate here:


```json
"t0Filter": {
    "id": "BasicCatalogFilter",
    "runConfig": {
        "catName": "milliquas",
        "catRAKey": "ra",
        "catDecKey": "dec",
        "searchMethod": "healpix",
        "rsArcsec": 3,
        "rejectOnMatch": true,
        "alertPosType": "av"
    }
```

The `rejectOnMatch` flag makes it possible to use catalog matching both to veto or to accept transients. That is, if you have a catalog of objects wich you are not (scientifically) interested in, you'd want to set the rejectOnMatch to True. In this case, a transitent will be accepted if no counterpart in the Boring Object Catalog is found. On the other hand, set rejectOnMatch to False if the catalog you're matching contains positoins of special interest to you. In this case, the alerts will be accepted if they match with the Really Cool Catalog.


To summarize the above settings: we will reject alerts if there is **at least one MQC** source within **3** arcsec from the all-epoch **average** position of the alert. The catalog ra/dec columns names are rather plain, and the queries routines will make use of the HEALPix index of the catalog.

------
**NOTE:**
The external catalogs to be examined have to be parsed into a database. The pacakage [extcats](https://github.com/MatteoGiomi/extcats) provides tools for setting up such a database and for querying it for positional matches. To run this example, git-clone the extcat package and run the setup script (the MQC is included in the test data).

In [1]:
# here some temp fixes
%env PYTHONPATH=../src/
import logging
logging.basicConfig(level=logging.INFO)

env: PYTHONPATH=../src/


## loading the channel:

In [ ]:
from ampel.pipeline.t0.AlertProcessor import AlertProcessor
ap=AlertProcessor(load_channels=False)
ap.load_channel("HU_NOAGN")

INFO:Ampel-12:32:01.508017:Setting up new AlertProcessor instance
INFO:Ampel-12:32:01.508017:AlertProcessor initial setup completed
INFO:Ampel-12:32:01.508017:Loading filter: BasicCatalogFilter
INFO:Ampel-12:32:01.508017:   Full class path: ampel.pipeline.t0.filters.BasicCatalogFilter
INFO:Ampel-12:32:01.508017:using mongo client at localhost:27017
INFO:Ampel-12:32:01.508017:connected to collection srcs of database milliquas.
INFO:Ampel-12:32:01.508017:found 1998464 documents in source collection srcs.
INFO:Ampel-12:32:01.508017:set HEALPIX partition of order 16 with key hpxid_16. Nested: True, Inexed: True, Resolution ["]: 3.221
INFO:Ampel-12:32:01.508017:set 2dsphere key pos with format geoJSON. Inexed: True
INFO:Ampel-12:32:01.508017:source collection has the following indexes: _i, hpxid_16, pos_2dsphe
INFO:Ampel-12:32:01.508017:Initialized BasicCatalogFilter. Alerts with matches in milliquas (rs = 5.00 arcsec) will be accepted
INFO:Ampel-12:32:01.508017:   Version: 0.1
INFO:Ampel-1

### explanation of the log (need Valery's insight, could be wrong, misleading, or both):

As we are interested in testing one specific channel, the Alert Processor is instanciated with the load_channels = False. This makes it possible to manually load just our HU_NOAGN channel.

The BasicCatalogfilter object is loaded, and it sucessfully connects to the MQC database. Some statistic on this database is printed; the database indexes are examined.

## Run it!

now we apply the filter to some tests alerts and measure the average execution time.

In [ ]:
import time, glob
test_alert_dir  = '../alerts/real'

start = time.time()
ap.run_iterable_paths(base_dir = test_alert_dir)
end = time.time()

tspan = end - start
nalerts = len(glob.glob(test_alert_dir+'/*.avro'))
tav = tspan / float(nalerts)
print ("took %.2e sec to process %d alerts. On average %.3e sec / alert"%
       (tspan, nalerts, tav))

INFO:Ampel-12:32:01.508017:Returning iterable for file paths in folder: ../alerts/real
INFO:Ampel-12:32:01.508017:Executing run method
INFO:Ampel-12:32:01.508017:#######     Processing alerts     #######
INFO:Ampel-12:32:01.508017:Configuring ZIAlertIngester with channels ['HU_NOAGN']
INFO:Ampel-12:32:01.508017:Processing alert: ZTF17aaaivxf
INFO:Ampel-12:32:01.508017: -> Channel 'HU_NOAGN': alert was rejected
INFO:Ampel-12:32:01.508017:Disposing rejected candidates not implemented yet
INFO:Ampel-12:32:01.508017:Processing alert: ZTF17aaaiuhi
INFO:Ampel-12:32:01.508017: -> Channel 'HU_NOAGN': alert was rejected
INFO:Ampel-12:32:01.508017:Disposing rejected candidates not implemented yet
INFO:Ampel-12:32:01.508017:Processing alert: ZTF17aaaiyiw
INFO:Ampel-12:32:01.508017: -> Channel 'HU_NOAGN': alert was rejected
INFO:Ampel-12:32:01.508017:Disposing rejected candidates not implemented yet
INFO:Ampel-12:32:01.508017:Processing alert: ZTF17aaaixva
INFO:Ampel-12:32:01.508017: -> Channel 'H

INFO:Ampel-12:32:01.508017:Processing alert: ZTF17aaaixyt
INFO:Ampel-12:32:01.508017: -> Channel 'HU_NOAGN': alert was rejected
INFO:Ampel-12:32:01.508017:Disposing rejected candidates not implemented yet
INFO:Ampel-12:32:01.508017:Processing alert: ZTF17aabtdps
INFO:Ampel-12:32:01.508017: -> Channel 'HU_NOAGN': alert was rejected
INFO:Ampel-12:32:01.508017:Disposing rejected candidates not implemented yet
INFO:Ampel-12:32:01.508017:Processing alert: ZTF17aaaiuuy
INFO:Ampel-12:32:01.508017: -> Channel 'HU_NOAGN': alert was rejected
INFO:Ampel-12:32:01.508017:Disposing rejected candidates not implemented yet
INFO:Ampel-12:32:01.508017:Processing alert: ZTF17aabtedn
INFO:Ampel-12:32:01.508017: -> Channel 'HU_NOAGN': alert was rejected
INFO:Ampel-12:32:01.508017:Disposing rejected candidates not implemented yet
INFO:Ampel-12:32:01.508017:Processing alert: ZTF17aabteao
INFO:Ampel-12:32:01.508017: -> Channel 'HU_NOAGN': alert was rejected
INFO:Ampel-12:32:01.508017:Disposing rejected candid

INFO:Ampel-12:32:01.508017:Processing alert: ZTF17aabtdzy
INFO:Ampel-12:32:01.508017: -> Channel 'HU_NOAGN': alert was rejected
INFO:Ampel-12:32:01.508017:Disposing rejected candidates not implemented yet
INFO:Ampel-12:32:01.508017:Processing alert: ZTF17aabtdsm
INFO:Ampel-12:32:01.508017: -> Channel 'HU_NOAGN': alert was rejected
INFO:Ampel-12:32:01.508017:Disposing rejected candidates not implemented yet
INFO:Ampel-12:32:01.508017:Processing alert: ZTF17aabtdqw
INFO:Ampel-12:32:01.508017: -> Channel 'HU_NOAGN': alert was rejected
INFO:Ampel-12:32:01.508017:Disposing rejected candidates not implemented yet
INFO:Ampel-12:32:01.508017:Processing alert: ZTF17aabtdzx
INFO:Ampel-12:32:01.508017: -> Channel 'HU_NOAGN': alert was rejected
INFO:Ampel-12:32:01.508017:Disposing rejected candidates not implemented yet
INFO:Ampel-12:32:01.508017:Processing alert: ZTF17aabtdsg
INFO:Ampel-12:32:01.508017: -> Channel 'HU_NOAGN': alert was rejected
INFO:Ampel-12:32:01.508017:Disposing rejected candid

INFO:Ampel-12:32:01.508017:Processing alert: ZTF17aabtdyt
INFO:Ampel-12:32:01.508017: -> Channel 'HU_NOAGN': alert was rejected
INFO:Ampel-12:32:01.508017:Disposing rejected candidates not implemented yet
INFO:Ampel-12:32:01.508017:Processing alert: ZTF17aabtdup
INFO:Ampel-12:32:01.508017: -> Channel 'HU_NOAGN': alert was rejected
INFO:Ampel-12:32:01.508017:Disposing rejected candidates not implemented yet
INFO:Ampel-12:32:01.508017:Processing alert: ZTF17aabtgki
INFO:Ampel-12:32:01.508017: -> Channel 'HU_NOAGN': alert was rejected
INFO:Ampel-12:32:01.508017:Disposing rejected candidates not implemented yet
INFO:Ampel-12:32:01.508017:Processing alert: ZTF17aabtebi
INFO:Ampel-12:32:01.508017: -> Channel 'HU_NOAGN': alert was rejected
INFO:Ampel-12:32:01.508017:Disposing rejected candidates not implemented yet
INFO:Ampel-12:32:01.508017:Processing alert: ZTF17aabtdyc
INFO:Ampel-12:32:01.508017: -> Channel 'HU_NOAGN': alert was rejected
INFO:Ampel-12:32:01.508017:Disposing rejected candid

INFO:Ampel-12:32:01.508017: -> Channel 'HU_NOAGN': alert was rejected
INFO:Ampel-12:32:01.508017:Disposing rejected candidates not implemented yet
INFO:Ampel-12:32:01.508017:Processing alert: ZTF17aabtdsl
INFO:Ampel-12:32:01.508017: -> Channel 'HU_NOAGN': alert was rejected
INFO:Ampel-12:32:01.508017:Disposing rejected candidates not implemented yet
INFO:Ampel-12:32:01.508017:Processing alert: ZTF17aabtelk
INFO:Ampel-12:32:01.508017: -> Channel 'HU_NOAGN': alert was rejected
INFO:Ampel-12:32:01.508017:Disposing rejected candidates not implemented yet
INFO:Ampel-12:32:01.508017:Processing alert: ZTF17aabtecg
INFO:Ampel-12:32:01.508017: -> Channel 'HU_NOAGN': alert was rejected
INFO:Ampel-12:32:01.508017:Disposing rejected candidates not implemented yet
INFO:Ampel-12:32:01.508017:Processing alert: ZTF17aabteyy
INFO:Ampel-12:32:01.508017: -> Channel 'HU_NOAGN': alert was rejected
INFO:Ampel-12:32:01.508017:Disposing rejected candidates not implemented yet
INFO:Ampel-12:32:01.508017:Proces

INFO:Ampel-12:32:01.508017: -> Channel 'HU_NOAGN': alert was rejected
INFO:Ampel-12:32:01.508017:Disposing rejected candidates not implemented yet
INFO:Ampel-12:32:01.508017:Processing alert: ZTF17aabteed
INFO:Ampel-12:32:01.508017: -> Channel 'HU_NOAGN': alert was rejected
INFO:Ampel-12:32:01.508017:Disposing rejected candidates not implemented yet
INFO:Ampel-12:32:01.508017:Processing alert: ZTF17aabteqd
INFO:Ampel-12:32:01.508017: -> Channel 'HU_NOAGN': alert was rejected
INFO:Ampel-12:32:01.508017:Disposing rejected candidates not implemented yet
INFO:Ampel-12:32:01.508017:Processing alert: ZTF17aabtfam
INFO:Ampel-12:32:01.508017: -> Channel 'HU_NOAGN': alert was rejected
INFO:Ampel-12:32:01.508017:Disposing rejected candidates not implemented yet
INFO:Ampel-12:32:01.508017:Processing alert: ZTF17aabtfaz
INFO:Ampel-12:32:01.508017: -> Channel 'HU_NOAGN': alert was rejected
INFO:Ampel-12:32:01.508017:Disposing rejected candidates not implemented yet
INFO:Ampel-12:32:01.508017:Proces

INFO:Ampel-12:32:01.508017: -> Channel 'HU_NOAGN': alert was rejected
INFO:Ampel-12:32:01.508017:Disposing rejected candidates not implemented yet
INFO:Ampel-12:32:01.508017:Processing alert: ZTF17aabtfdz
INFO:Ampel-12:32:01.508017: -> Channel 'HU_NOAGN': alert was rejected
INFO:Ampel-12:32:01.508017:Disposing rejected candidates not implemented yet
INFO:Ampel-12:32:01.508017:Processing alert: ZTF17aabtesh
INFO:Ampel-12:32:01.508017: -> Channel 'HU_NOAGN': alert was rejected
INFO:Ampel-12:32:01.508017:Disposing rejected candidates not implemented yet
INFO:Ampel-12:32:01.508017:Processing alert: ZTF17aabtfef
INFO:Ampel-12:32:01.508017: -> Channel 'HU_NOAGN': alert was rejected
INFO:Ampel-12:32:01.508017:Disposing rejected candidates not implemented yet
INFO:Ampel-12:32:01.508017:Processing alert: ZTF17aabtetw
INFO:Ampel-12:32:01.508017: -> Channel 'HU_NOAGN': alert was rejected
INFO:Ampel-12:32:01.508017:Disposing rejected candidates not implemented yet
INFO:Ampel-12:32:01.508017:Proces